In [ ]:
%load_ext autoreload
%autoreload 2

import os
import time
import pickle
import numpy as np
import multiprocessing as mp
from IPython.display import display, Image, SVG

from pydrake.geometry import CollisionFilterDeclaration, GeometrySet, Role, SceneGraph
from pydrake.geometry.optimization import (
    HPolyhedron,
    Hyperellipsoid,
    IrisInConfigurationSpace,
    IrisOptions
)
from pydrake.math import RollPitchYaw, RotationMatrix
from pydrake.multibody.inverse_kinematics import InverseKinematics
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, MultibodyPlant
from pydrake.multibody.tree import RevoluteJoint
from pydrake.solvers.mathematicalprogram import Solve
from pydrake.solvers.gurobi import GurobiSolver
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder, LeafSystem
from pydrake.systems.meshcat_visualizer import ConnectMeshcatVisualizer
from pydrake.systems.primitives import TrajectorySource
from pydrake.systems.rendering import MultibodyPositionToGeometryPose

from meshcat import Visualizer
import meshcat.geometry as g

from comparison.helpers import make_traj
from spp.bezier import BezierSPP
from spp.linear import LinearSPP

GurobiSolver.AcquireLicense()
MosekSolver.AcquireLicense()

In [ ]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Sporadically need to run `pkill -f meshcat`

In [ ]:
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant)
parser.package_map().Add("mp-gcs", os.path.dirname("./package.xml"))

directives_file = "./models/bimanual_iiwa.yaml"
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)
[iiwa_1, wsg_1, iiwa_2, wsg_2, shelf, bin_1, bin_2, table] = models


plant.Finalize()

viz_role = Role.kIllustration
# viz_role = Role.kProximity
visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url,
                                      delete_prefix_on_load=False, role=viz_role)

diagram = builder.Build()

q0 = [0.0, -0.2, 0, -1.2, 0, 1.6, 0.0]
index = 0
for joint_index in plant.GetJointIndices(iiwa_1.model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1
index = 0
for joint_index in plant.GetJointIndices(iiwa_2.model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1
        
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)
sg_context = scene_graph.GetMyMutableContextFromRoot(context)
diagram.Publish(context)

In [ ]:
filter_manager = scene_graph.collision_filter_manager(sg_context)
inspector = scene_graph.model_inspector()

iiwa1 = [[], [], [], [], [], [], [], []]
iiwa2 = [[], [], [], [], [], [], [], []]
wsg1 = []
wsg2 = []
shelf = []
bins = [[], []]
table = []
for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity):
    gid_name = inspector.GetName(inspector.GetFrameId(gid))
    if "iiwa_1::iiwa_link_" in gid_name:
        link_num = gid_name[18]
        iiwa1[int(link_num)].append(gid)
    elif "iiwa_2::iiwa_link_" in gid_name:
        link_num = gid_name[18]
        iiwa2[int(link_num)].append(gid)
    elif "wsg_1" in gid_name:
        wsg1.append(gid)
    elif "wsg_2" in gid_name:
        wsg2.append(gid)
    elif "shelves::" in gid_name:
        shelf.append(gid)
    elif "binR" in gid_name:
        bins[0].append(gid)
    elif "binL" in gid_name:
        bins[1].append(gid)
    elif "table" in gid_name:
        table.append(gid)
    else:
        print(gid_name)


filter_manager.Apply(CollisionFilterDeclaration().ExcludeWithin(GeometrySet(
        iiwa1[0] + iiwa1[1] + iiwa1[2] + iiwa1[3] + shelf)))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[1] + iiwa1[2]+ iiwa1[3]), GeometrySet(iiwa1[4] + iiwa1[5])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[3] + iiwa1[4]), GeometrySet(iiwa1[6])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[2] + iiwa1[3] + iiwa1[4] + iiwa1[5] + iiwa1[6]), GeometrySet(iiwa1[7] + wsg1)))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[0] + iiwa1[0] + iiwa1[2]), GeometrySet(bins[0])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[0] + iiwa1[1] + iiwa1[2] + iiwa1[3] + iiwa1[4]), GeometrySet(bins[1])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[0] + iiwa1[0] + iiwa1[2]), GeometrySet(table)))

filter_manager.Apply(CollisionFilterDeclaration().ExcludeWithin(GeometrySet(
        iiwa2[0] + iiwa2[1] + iiwa2[2] + iiwa2[3] + shelf)))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa2[1] + iiwa2[2]+ iiwa2[3]), GeometrySet(iiwa2[4] + iiwa2[5])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa2[3] + iiwa2[4]), GeometrySet(iiwa2[6])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa2[2] + iiwa2[3] + iiwa2[4] + iiwa2[5] + iiwa2[6]), GeometrySet(iiwa2[7] + wsg2)))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa2[0] + iiwa2[0] + iiwa2[2]), GeometrySet(bins[1])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa2[0] + iiwa2[1] + iiwa2[2] + iiwa2[3] + iiwa2[4]), GeometrySet(bins[0])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa2[0] + iiwa2[0] + iiwa2[2]), GeometrySet(table)))

filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[0] + iiwa1[1]), GeometrySet(iiwa2[0] + iiwa2[1])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[2]), GeometrySet(iiwa2[0] + iiwa2[1])))
filter_manager.Apply(CollisionFilterDeclaration().ExcludeBetween(
        GeometrySet(iiwa1[0] + iiwa1[1]), GeometrySet(iiwa2[2])))

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

In [ ]:
ik_points = {
    "top_shelf/top_shelf": ([0.7, 0.15, 0.9], [-np.pi+0.1, 0, np.pi/2], [0.7, 0.35, 0.9], [-np.pi+0.1, 0, np.pi/2]),
    "top_shelf/shelf_1": ([0.7, 0.15, 0.9], [-np.pi+0.1, 0, np.pi/2], [0.7, 0.35, 0.65], [-np.pi+0.1, 0, np.pi/2]),
    "top_shelf/shelf_2": ([0.7, 0.15, 0.9], [-np.pi+0.1, 0, np.pi/2], [0.7, 0.35, 0.4], [-np.pi+0.1, 0, np.pi/2]),
    "top_shelf/bin_L": ([0.7, 0.15, 0.9], [-np.pi+0.1, 0, np.pi/2], [0., 1.1, 0.3], [-np.pi/2+0.1, 0, np.pi]),
    
    "shelf_1/top_shelf": ([0.7, 0.15, 0.65], [-np.pi+0.1, 0, np.pi/2], [0.7, 0.35, 0.9], [-np.pi+0.1, 0, np.pi/2]),
    "shelf_1/shelf_1": ([0.7, 0.15, 0.65], [-np.pi+0.1, 0, np.pi/2], [0.7, 0.35, 0.65], [-np.pi+0.1, 0, np.pi/2]),
    "shelf_1/shelf_2": ([0.7, 0.15, 0.65], [-np.pi+0.1, 0, np.pi/2], [0.7, 0.35, 0.4], [-np.pi+0.1, 0, np.pi/2]),
    "shelf_1/bin_L": ([0.7, 0.15, 0.65], [-np.pi+0.1, 0, np.pi/2], [0., 1.1, 0.3], [-np.pi/2+0.1, 0, np.pi]),
    
    "shelf_2/top_shelf": ([0.7, 0.15, 0.4], [-np.pi+0.1, 0, np.pi/2], [0.7, 0.35, 0.9], [-np.pi+0.1, 0, np.pi/2]),
    "shelf_2/shelf_1": ([0.7, 0.15, 0.4], [-np.pi+0.1, 0, np.pi/2], [0.7, 0.35, 0.65], [-np.pi+0.1, 0, np.pi/2]),
    "shelf_2/shelf_2": ([0.7, 0.15, 0.4], [-np.pi+0.1, 0, np.pi/2], [0.7, 0.35, 0.4], [-np.pi+0.1, 0, np.pi/2]),
    "shelf_2/bin_L": ([0.7, 0.15, 0.4], [-np.pi+0.1, 0, np.pi/2], [0., 1.1, 0.3], [-np.pi/2+0.1, 0, np.pi]),
    
    "bin_R/top_shelf": ([0.0, -0.6, 0.3], [-np.pi/2+0.1, 0, -np.pi], [0.7, 0.35, 0.9], [-np.pi+0.1, 0, np.pi/2]),
    "bin_R/shelf_1": ([0.0, -0.6, 0.3], [-np.pi/2+0.1, 0, -np.pi], [0.7, 0.35, 0.65], [-np.pi+0.1, 0, np.pi/2]),
    "bin_R/shelf_2": ([0.0, -0.6, 0.3], [-np.pi/2+0.1, 0, -np.pi], [0.7, 0.35, 0.4], [-np.pi+0.1, 0, np.pi/2]),
    "bin_R/bin_L": ([0.0, -0.6, 0.3], [-np.pi/2+0.1, 0, -np.pi], [0., 1.1, 0.3], [-np.pi/2+0.1, 0, np.pi]),

    "top_shelf/shelf_1_extract": ([0.7, 0.15, 0.9], [-np.pi+0.1, 0, np.pi/2], [0.5, 0.35, 0.65], [-np.pi+0.1, 0, np.pi/2]),
    "top_shelf/shelf_2_extract": ([0.7, 0.15, 0.9], [-np.pi+0.1, 0, np.pi/2], [0.5, 0.35, 0.4], [-np.pi+0.1, 0, np.pi/2]),
    "shelf_2_extract/top_shelf": ([0.5, 0.15, 0.4], [-np.pi+0.1, 0, np.pi/2], [0.7, 0.35, 0.9], [-np.pi+0.1, 0, np.pi/2]),
    "shelf_1_extract/top_shelf": ([0.5, 0.15, 0.65], [-np.pi+0.1, 0, np.pi/2], [0.7, 0.35, 0.9], [-np.pi+0.1, 0, np.pi/2]),
    
    "top_shelf/shelf_1_cross": ([0.7, 0.15, 0.9], [-np.pi+0.1, 0, np.pi/2], [0.7, 0.15, 0.65], [-np.pi+0.1, 0, np.pi/2-0.3]),
    "cross_table/top_shelf_cross": ([0.4, 0.4, 0.2], [-np.pi+0.1, 0, np.pi], [0.7, 0.15, 0.9], [-np.pi+0.1, 0, np.pi/2]),
    "shelf_2_cross/top_shelf_cross": ([0.7, 0.35, 0.4], [-np.pi+0.1, 0, np.pi/2+0.4], [0.7, 0.15, 0.9], [-np.pi+0.1, 0, np.pi/2-0.4]),
}

ik_seed = ik_points["shelf_2_cross/top_shelf_cross"]

hand_frame1 = plant.GetBodyByName("body", wsg_1.model_instance).body_frame()
hand_frame2 = plant.GetBodyByName("body", wsg_2.model_instance).body_frame()

ref_config = np.concatenate((q0, q0))
# ref_config = seed_points["top_shelf/shelf_1"]
ik = InverseKinematics(plant, plant_context)
# ik.AddMinimumDistanceConstraint(0.01, 0.01)
ik.prog().AddBoundingBoxConstraint(plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits(), ik.q())
ik.prog().SetInitialGuess(ik.q(), ref_config)
ik.prog().AddQuadraticCost((ik.q() - ref_config).dot(ik.q() - ref_config))

ik.AddPositionConstraint(hand_frame1, [0, 0, 0], plant.world_frame(), ik_seed[0], ik_seed[0])
ik.AddOrientationConstraint(hand_frame1, RotationMatrix(), plant.world_frame(),
                            RotationMatrix(RollPitchYaw(*ik_seed[1])), 0.001)

ik.AddPositionConstraint(hand_frame2, [0, 0, 0], plant.world_frame(), ik_seed[2], ik_seed[2])
ik.AddOrientationConstraint(hand_frame2, RotationMatrix(), plant.world_frame(),
                            RotationMatrix(RollPitchYaw(*ik_seed[3])), 0.001)

result = Solve(ik.prog())
viz_q = result.GetSolution(ik.q())
print(repr(viz_q))

plant.SetPositions(plant_context, viz_q)
diagram.Publish(context)

In [ ]:
seed_points = {
    "top_shelf/top_shelf": [ 0.37080011,  0.41394084, -0.16861973, -0.70789778, -0.37031516, 0.60412162,  0.39982981, -0.37080019,  0.41394089,  0.16861988, -0.70789766,  0.37031506,  0.60412179, -0.39982996],
    "top_shelf/shelf_1": [ 0.37080079,  0.41394132, -0.16862043, -0.70789679, -0.37031656, 0.60412327,  0.39982969, -0.93496924,  0.46342534,  0.92801666, -1.45777635, -0.31061724, -0.0657716 , -0.06019899],
    "top_shelf/shelf_2": [ 0.37086448,  0.41394538, -0.16875166, -0.70789745, -0.37020563, 0.60411217,  0.399785  , -0.4416204 ,  0.62965228,  0.20598405, -1.73324339, -0.41354372, -0.68738414,  0.17443976],
    "top_shelf/bin_L": [ 0.37081989,  0.41394235, -0.16866012, -0.70789737, -0.37028201, 0.60411923,  0.39981634, -0.89837331, -1.1576151 ,  1.75505216, -1.37515153,  1.0676443 ,  1.56371166, -0.64126346],
    
    "shelf_1/top_shelf": [ 0.93496924,  0.46342534, -0.92801666, -1.45777635,  0.31061724, -0.0657716 ,  0.06019899, -0.37080079,  0.41394132,  0.16862043, -0.70789679,  0.37031656,  0.60412327, -0.39982969],
    "shelf_1/shelf_1": [ 0.87224109,  0.43096634, -0.82223436, -1.45840049,  0.73813452, -0.08999384, -0.41624203, -0.87556489,  0.43246906,  0.82766047, -1.45838515, -0.72259842, -0.0884963 ,  0.39840129],
    "shelf_1/shelf_2": [ 0.93496866,  0.463425  , -0.92801564, -1.45777634,  0.3106235, -0.06577172,  0.06019173, -0.44158858,  0.62964838,  0.20594112, -1.73324341, -0.41354987, -0.6873923 ,  0.17446778],
    "shelf_1/bin_L": [ 0.93496918,  0.46342531, -0.92801656, -1.45777637,  0.31061728, -0.06577167,  0.06019927, -0.89837321, -1.15761746,  1.75504915, -1.37515113,  1.06764716,  1.56371454, -0.64126383],
    
    "shelf_2/top_shelf": [ 0.4416204,  0.62965228, -0.20598405, -1.73324339,  0.41354372, -0.68738414, -0.17443976, -0.37086448,  0.41394538,  0.16875166, -0.70789745,  0.37020563,  0.60411217, -0.399785  ],
    "shelf_2/shelf_1": [ 0.44158858,  0.62964838, -0.20594112, -1.73324341,  0.41354987, -0.6873923, -0.17446778, -0.93496866,  0.463425  ,  0.92801564, -1.45777634, -0.3106235 , -0.06577172, -0.06019173],
    "shelf_2/shelf_2": [ 0.44161313,  0.62965141, -0.20597435, -1.73324346,  0.41354447, -0.68738613, -0.17444557, -0.4416132 ,  0.62965142,  0.20597452, -1.73324348, -0.41354416, -0.68738609,  0.17444625],
    "shelf_2/bin_L": [ 0.44161528,  0.62965169, -0.20597726, -1.73324347,  0.41354399, -0.68738565, -0.17444283, -1.37292761, -0.68372976,  2.96705973, -1.41521783,  2.96705973, -1.11343251, -3.0140737 ],
    
    "bin_R/top_shelf": [ 0.81207926, -1.25359738, -1.58098625, -1.5155474 , -1.32223687, 1.50549708, -2.38221725, -0.37085114,  0.4139444 ,  0.16872443, -0.70789757,  0.37022786,  0.60411401, -0.39979449],
    "bin_R/shelf_1": [ 0.81207923, -1.25358454, -1.58100042, -1.51554769, -1.32222337, 1.50548369, -2.3822204 , -0.9349716 ,  0.46342674,  0.92802082, -1.45777624, -0.31059455, -0.0657707 , -0.06022391],
    "bin_R/shelf_2": [ 0.81207937, -1.25360462, -1.58097816, -1.51554761, -1.32224557, 1.50550485, -2.38221483, -0.44166552,  0.62965782,  0.20604497, -1.7332434 , -0.41353464, -0.6873727 ,  0.17439863],
    "bin_R/bin_L": [-1.73637519,  0.6209681 ,  0.24232887, -1.51538355, -0.17977474, 0.92618894, -3.01360257,  1.31861497,  0.72394333,  0.4044295 , -1.37509496, -0.27461997,  1.20038493,  0.18611701],
    
    "neutral/neutral": [ 0.0, -0.2, 0, -1.2, 0, 1.6, 0.0, 0.0, -0.2, 0, -1.2, 0, 1.6, 0.0],

    "neutral/shelf_1": [ 0.0, -0.2, 0, -1.2, 0, 1.6, 0.0, -0.93496866,  0.463425  ,  0.92801564, -1.45777634, -0.3106235 , -0.06577172, -0.06019173],
    "neutral/shelf_2": [ 0.0, -0.2, 0, -1.2, 0, 1.6, 0.0, -0.44166552,  0.62965782,  0.20604497, -1.7332434 , -0.41353464, -0.6873727 ,  0.17439863],
    "shelf_1/neutral": [ 0.93496924,  0.46342534, -0.92801666, -1.45777635,  0.31061724, -0.0657716 ,  0.06019899, 0.0, -0.2, 0, -1.2, 0, 1.6, 0.0],
    "shelf_2/neutral": [ 0.44161528,  0.62965169, -0.20597726, -1.73324347,  0.41354399, -0.68738565, -0.17444283, 0.0, -0.2, 0, -1.2, 0, 1.6, 0.0],
    
    "shelf_2_cross/top_shelf_cross": [ 0.47500706,  0.72909874,  0.01397772, -1.52841372,  0.15392366, -0.591641  , -0.12870521, -0.48821156,  0.67762534,  0.02049926, -0.27420758,  0.10620709,  0.72215209, -0.09973172],
}

#  Additional seed points not needed to connect the graph
#     "neutral/shelf_1_extract": [ 0.0, -0.2, 0, -1.2, 0, 1.6, 0.0, -0.35486829, -0.10621117, -0.09276445, -1.94995786,  1.88826556,  0.46922151, -1.98267349],
#     "neutral/shelf_2_extract": [ 0.0, -0.2, 0, -1.2, 0, 1.6, 0.0, 0.3078069 ,  0.56765359, -0.86829439, -2.0943951 ,  2.53950045,  1.09607546, -2.4169564],
#     "shelf_1_extract/neutral": [-1.05527083, -0.43710629,  1.15648812, -1.95011062,  0.24422131, -0.07820216,  0.15872416, 0.0, -0.2, 0, -1.2, 0, 1.6, 0.0],
#     "shelf_2_extract/neutral": [-0.30739053,  0.5673891 ,  0.86772198, -2.0943951 , -2.53946773, 1.09586777,  2.41729532, 0.0, -0.2, 0, -1.2, 0, 1.6, 0.0],
#     "cross_table/top_shelf_cross": [ 0.04655887,  0.97997658,  0.52004246, -1.91926412, -1.37518707, -0.88823968,  0.07674699, -0.5921624 ,  0.83651867,  0.20513136, -0.00257881,  0.51748756,  0.92012332, -0.51686487],

In [ ]:
iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 5
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.02
iris_options.enable_ibex = False
CORE_CNT = 10  # mp.cpu_count() # you may edit this

In [ ]:
context_clones = [context.Clone() for _ in range(len(seed_points))]

def calcRegion(ii, seed):
    start_time = time.time()
    context = context_clones[ii]
    plant_context = plant.GetMyMutableContextFromRoot(context)
    plant.SetPositions(plant_context, seed)
    
    hpoly = IrisInConfigurationSpace(plant, plant_context, iris_options)
    print("Seed:", ii, "\tTime:", np.round((time.time() - start_time)/60., 4),
          "minutes.\tFaces", len(hpoly.b()), flush=True)
    return hpoly

def generateRegions(seed_points):
    seeds = list(seed_points.values()) if type(seed_points) is dict else seed_points
    regions = []
    loop_time = time.time()
    with mp.Pool(processes = CORE_CNT) as pool:
        regions = pool.starmap(calcRegion, [[ii, seed] for ii, seed in enumerate(seeds)])

    print("Loop time:", time.time() - loop_time)
    
    if type(seed_points) is dict:
        return dict(list(zip(seed_points.keys(), regions)))

    return regions
    
regions.update(generateRegions(seed_points))

In [ ]:
with open('./data/bimanual/iris_regions.reg', 'wb') as f:
    pickle.dump(regions,f)

In [ ]:
with open('data/bimanual/iris_regions.reg', 'rb') as f:
    regions = pickle.load(f)

In [ ]:
spp = LinearSPP(regions)
display(Image(spp.VisualizeGraph("png")))

# Demonstration

In [ ]:
def getLinearSppPath(regions, sequence):
    spp = LinearSPP(regions)
    spp.setPaperSolverOptions()
    spp.setSolver(MosekSolver())
    path = [sequence[0]]
    run_time = 0.0
    for start_pt, goal_pt in zip(sequence[:-1], sequence[1:]):
        start_time = time.time()
        waypoints, result_relax, best_result_relax, hard_result_relax = spp.SolvePath(start_pt, goal_pt, True, False)
        if waypoints is None:
            print(f"Failed between {start_pt} and {goal_pt}")
            return None
        print(f"Planned segment in {np.round(time.time() - start_time, 4)}", flush=True)
        run_time += result_relax.get_solver_details().optimizer_time
        max_hard_result_time = 0
        for result in hard_result_relax:
            if result.get_solver_details().optimizer_time > max_hard_result_time:
                max_hard_result_time = result.get_solver_details().optimizer_time
        run_time += max_hard_result_time
        
        path += waypoints.T[1:].tolist()
    
    return np.stack(path).T, run_time

def getBezierSppPath(regions, sequence, order, continuity, hdot_min = 1e-3):
    spp = BezierSPP(regions, order, continuity)
    spp.addTimeCost(1)
    spp.addPathLengthCost(1)
    spp.addAccelerationRegularization(1e-3, 1e-3)
    spp.addVelocityLimits(0.6*plant.GetVelocityLowerLimits(), 0.6*plant.GetVelocityUpperLimits())
    spp.setPaperSolverOptions()
    spp.setSolver(MosekSolver())
    
    run_time = 0.0
    trajectories = []
    for start_pt, goal_pt in zip(sequence[:-1], sequence[1:]):
        start_time = time.time()
        segment_traj, result, best_result, hard_result = spp.SolvePath(start_pt, goal_pt, True, False)
        if segment_traj is None:
            print(f"Failed between {start_pt} and {goal_pt}")
            return None
        print(f"Planned segment in {np.round(time.time() - start_time, 4)}", flush=True)
        run_time += result.get_solver_details().optimizer_time
        max_hard_result_time = 0
        for r in hard_result:
            if r.get_solver_details().optimizer_time > max_hard_result_time:
                max_hard_result_time = r.get_solver_details().optimizer_time
        run_time += max_hard_result_time
        trajectories.append(segment_traj)
        
    return trajectories, run_time

class VectorTrajectorySource(LeafSystem):
    def __init__(self, trajectories):
        LeafSystem.__init__(self)
        self.trajectories = trajectories
        self.start_time = [0]
        for traj in trajectories:
            self.start_time.append(self.start_time[-1] + traj.end_time())
        self.start_time = np.array(self.start_time)
        self.port = self.DeclareVectorOutputPort("traj_eval", 14, self.DoVecTrajEval, {self.time_ticket()})
    
    def DoVecTrajEval(self, context, output):
        t = context.get_time()
        traj_index = np.argmax(self.start_time > t) - 1
        
        q = self.trajectories[traj_index].value(t - self.start_time[traj_index])
        output.set_value(q)

def visualize_trajectory(traj):
    builder = DiagramBuilder()
    
    scene_graph = builder.AddSystem(SceneGraph())
    plant = MultibodyPlant(time_step=0.0)
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)
    parser.package_map().Add("mp-gcs", os.path.dirname("./package.xml"))

    directives_file = "./models/bimanual_iiwa.yaml"
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)
    [iiwa_1, wsg_1, iiwa_2, wsg_2, shelf, binR, binL, table] =  models
    
    plant.Finalize()

    to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
    builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))

    if type(traj) is list:
        traj_system = builder.AddSystem(VectorTrajectorySource(traj))
        end_time = np.sum([t.end_time() for t in traj])
    else:
        traj_system = builder.AddSystem(TrajectorySource(traj))
        end_time = traj.end_time()
    builder.Connect(traj_system.get_output_port(), to_pose.get_input_port())
    
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)

    vis_diagram = builder.Build()
    simulator = Simulator(vis_diagram)
    
    plant_context = plant.CreateDefaultContext()
    c_list_rgb = [[i/255 for i in (0, 0, 255, 255)],[i/255 for i in (255, 191, 0, 255)],[i/255 for i in (255, 64, 0, 255)]]
    iiwa1_X = []
    iiwa2_X = []
    if type(traj) is list:
        for t in traj:
            q_waypoints = t.vector_values(np.linspace(t.start_time(), t.end_time(), 1000))
            for ii in range(q_waypoints.shape[1]):
                plant.SetPositions(plant_context, q_waypoints[:, ii])
                iiwa1_X.append(plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body", wsg_1.model_instance)))
                iiwa2_X.append(plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body", wsg_2.model_instance)))

        vertices = list(map(lambda X: X.translation(), iiwa1_X))
        colors = [np.array(c_list_rgb[0]) for _ in range(len(iiwa1_X))]
        vertices = np.stack(vertices).astype(np.float32).T
        colors = np.array(colors).astype(np.float32).T
        vis["paths"]["iiwa_1"].set_object(g.Points(g.PointsGeometry(vertices, color=colors),
                                        g.PointsMaterial(size=0.015)))
        vertices = list(map(lambda X: X.translation(), iiwa2_X))
        colors = [np.array(c_list_rgb[1]) for _ in range(len(iiwa2_X))]
        vertices = np.stack(vertices).astype(np.float32).T
        colors = np.array(colors).astype(np.float32).T
        vis["paths"]["iiwa_2"].set_object(g.Points(g.PointsGeometry(vertices, color=colors),
                                        g.PointsMaterial(size=0.015)))
    
    meshcat.start_recording()
    simulator.AdvanceTo(end_time)
    meshcat.publish_recording()

In [ ]:
demo = [seed_points["neutral/neutral"],
        seed_points["shelf_1/shelf_1"],
        seed_points["shelf_2_cross/top_shelf_cross"],
        seed_points["bin_R/bin_L"],
]

In [ ]:
waypoints, run_time = getLinearSppPath(regions, demo)
print("Trajectory generation took", np.round(run_time, 4), "seconds of optimizer time.")
linear_spp_traj = make_traj(waypoints, speed = 2)
visualize_trajectory([linear_spp_traj])

with open ("data/bimanual/linear_trajectory.html", "w") as f:
   f.write(vis.static_html())

In [ ]:
trajectories, run_time = getBezierSppPath(regions, demo, 3, 1)
print("Trajectory generation took", np.round(run_time, 4), "seconds of optimizer time.")
visualize_trajectory(trajectories)

with open ("data/bimanual/bezier_trajectory.html", "w") as f:
   f.write(vis.static_html())